![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


   # **Cortex Game: Round1--Amount**

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**1- Make sure that your Python version is 3.3 or higher**

In [1]:
from platform import python_version
print (python_version())

3.9.13


**2- Install SASPy**

In [2]:
pip install saspy

Note: you may need to restart the kernel to use updated packages.


**3- Make sure that the configuration file "sascfg_personal.py" is correctly created**

In [3]:
import saspy, os
print(saspy.__file__.replace('__init__.py', 'sascfg_personal.py'))


C:\Users\CarlosD\anaconda3\lib\site-packages\saspy\sascfg_personal.py


**4- Establish Connection (Need to do this step each time you use SASPy)**

In [4]:
import saspy
sas_session = saspy.SASsession()
sas_session
;

Using SAS Config named: oda
SAS Connection established. Subprocess id is 14164



''

***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [5]:
%%SAS sas_session
libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';


## Transform cloud SAS dataset to Python dataframe (pandas)


> **For reference**:

> 1. [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2. [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)


In [6]:
import pandas as pd

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd1',
libref='cortex'
)

## Merge the Data

In [7]:
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge.sample(2)

#data_merge.head()


,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,GaveThisYear,AmtThisYear
77392,2077393.0,HAIRSTON,JOSEPH,0.0,19.0,11900.0,University / College,Suburban,4.0,1.0,...,3.0,1.0,3.0,20.0,20.0,20.0,0.0,0.0,0.0,0.0
562275,2562276.0,CAVANAUGH,RICKEY,0.0,39.0,192700.0,University / College,Suburban,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


## Treat Missing Values

> Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.


In [8]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['MaxGift']] = data_merge[['MinGift']].fillna(value=0)
data_merge[['TotalGift']] = data_merge[['MinGift']].fillna(value=0)

data_merge.City.value_counts()

City        332657
Suburban    312671
Rural       197978
Downtown    156694
Name: City, dtype: int64

In [9]:
data_merge.loc[data_merge["Education"] == "Elementary", "Education"] = 0

data_merge.loc[data_merge["Education"] == "High School", "Education"] = 1

data_merge.loc[data_merge["Education"] == "University / College", "Education"] = 2

dummies = pd.get_dummies(data_merge['City'], drop_first = True)
#dummies.head()

# Añadimos las variables binarias al DataFrame
data_merge = pd.concat([data_merge, dummies], axis = 1)

# Eliminamos la vairable original City
data_merge = data_merge.drop(columns=['City'])

data_merge.sample(3)

,ID,LastName,FirstName,Woman,Age,Salary,Education,SeniorList,NbActivities,Referrals,...,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,GaveThisYear,AmtThisYear,Downtown,Rural,Suburban
148322,2148323.0,BURLESON,ALLISON,1.0,55.0,700.0,2,3.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
889401,2889402.0,VANSLOOTEN,SUSANA,1.0,26.0,143900.0,2,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,20.0,0,0,1
205789,2205790.0,WOOD,ARTHUR,0.0,33.0,40100.0,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1


## Data Partition

In [10]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.4, random_state=12345) 

#train.head()
train.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,SeniorList,NbActivities,Referrals,...,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,GaveThisYear,AmtThisYear,Downtown,Rural,Suburban
885384,2885385.0,ROMERO,SHERRILL,1.0,19.0,11200.0,2,8.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
125105,2125106.0,BALL,JAMES,0.0,50.0,183400.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


## Prebuilt Models
***

### **Linear Regression Model**


> The [sk-learn library](https://scikit-learn.org/stable/index.html ) offers more advanced models.


In [11]:
from sklearn import linear_model

#comment: it's numpy array
X_train = train[['Age', 'Salary','MinGift', 'AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','MinGift', 'AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()
regr.fit(X_train,Y_train)
regr_predict=regr.predict(X_valid)

In [12]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

13.237223613998076
7597.251704507017
87.16221488986507


## **Regression Tree Model**

In [13]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age', 'Salary','MinGift', 'AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','MinGift', 'AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=5).fit(X_train,Y_train)

DT_predict = DT_model.predict(X_valid) #Predictions on Testing data


In [14]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

13.172511895904474
7617.65032281849
87.27915170771591


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html

***


## Scoring New Data

### Prepare data for scoring

In [15]:
data3 = sas_session.sasdata2dataframe(
table='score_rd1',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)

 ### Score new data based on your champion model
 
> Pick your champion model from previous steps and use it to predict next year donations. 
 
> In this case, the linear regression model performed better than the regression tree based on the MSE criterion.

In [16]:
scoring_data = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data[['MinGift']] = scoring_data[['MinGift']].fillna(value=0)
scoring_data[['MaxGift']] = scoring_data[['MinGift']].fillna(value=0)
scoring_data[['TotalGift']] = scoring_data[['MinGift']].fillna(value=0)

scoring_data.loc[scoring_data["Education"] == "Elementary", "Education"] = 0

scoring_data.loc[scoring_data["Education"] == "High School", "Education"] = 1

scoring_data.loc[scoring_data["Education"] == "University / College", "Education"] = 2

dummies = pd.get_dummies(scoring_data['City'], drop_first = True)
#dummies.head()

# Añadimos las variables binarias al DataFrame
scoring_data = pd.concat([scoring_data, dummies], axis = 1)

# Eliminamos la vairable original City
scoring_data = scoring_data.drop(columns=['City'])

scoring_data.head()

,ID,GaveLastYear,AmtLastYear,LastName,FirstName,Woman,Age,Salary,Education,SeniorList,...,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,Downtown,Rural,Suburban
0,2000001.0,0.0,0.0,ROMMES,RODNEY,0.0,25.0,107200.0,2,2.0,...,0.0,1.0,2.0,2.0,10.0,10.0,10.0,0,0,0
1,2000002.0,0.0,0.0,RAMIREZ,SHARON,1.0,38.0,15800.0,1,4.0,...,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0
2,2000003.0,0.0,0.0,TSOSIE,KAREN,1.0,37.0,57400.0,2,5.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0
3,2000004.0,0.0,0.0,LEE,MARY,1.0,78.0,23700.0,1,3.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0
4,2000005.0,0.0,0.0,HUMPHRES,ANGIE,1.0,34.0,71900.0,2,8.0,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0,1,0


In [17]:
# In this case, based on MSE (Mean Squared Error) criterion,
# the linear regression model performed better than the regression tree.

X = scoring_data[['Age', 'Salary','MinGift', 'AmtLastYear','Woman', 'NbActivities', 'Education', 'Downtown', 'Rural', 'Suburban', 'SeniorList', 'Referrals', 'MaxGift', 'TotalGift']] 
regr_predict_end=regr.predict(X)

scoring_data['Prediction'] = regr_predict_end
scoring_data.sort_values(by=['Prediction'], inplace=True,ascending=False)
scoring_data.head()

,ID,GaveLastYear,AmtLastYear,LastName,FirstName,Woman,Age,Salary,Education,SeniorList,...,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,Downtown,Rural,Suburban,Prediction
420890,2420891.0,1.0,9000.0,BEIL,MARGARET,1.0,37.0,104200.0,0,10.0,...,7.0,1.0,7.0,7000.0,7000.0,7000.0,1,0,0,198.351716
334249,2334250.0,1.0,10000.0,MANLEY,COLLEEN,1.0,36.0,108300.0,2,6.0,...,5.0,2.0,6.0,20.0,20.0,20.0,0,0,1,143.641458
631673,2631674.0,1.0,10000.0,KOPPENHEFFER,JENNIFER,1.0,34.0,186500.0,2,9.0,...,0.0,1.0,0.0,500.0,500.0,500.0,0,0,0,139.480369
416110,2416111.0,1.0,10000.0,GOLDSTEIN,MICHELLE,1.0,27.0,46900.0,2,5.0,...,1.0,3.0,3.0,50.0,50.0,50.0,0,0,0,135.046138
132410,2132411.0,1.0,9000.0,ASANTE,SOLEDAD,1.0,36.0,180500.0,1,7.0,...,2.0,4.0,7.0,10.0,10.0,10.0,0,0,1,133.380455


## Exporting Results to a CSV File

In [18]:
Result= scoring_data[['ID','Prediction']]
#Result.to_csv('Round1_Output.csv', index=False)

In [19]:
# Define your cutoff and choose a number of rows to submit to the leaderboard

NB = 200000
submission = Result.head(NB)
submission.to_csv('Round1 Output.csv', index=False)

In [20]:
# Reminder: Please note that you need only one column (the list of donors' IDs) to submit to the leaderboard.


In [21]:
Result1= Result['ID']
NB = 200000
submission = Result1.head(NB)
submission.to_csv('Round1 Output IDs.csv', index=False)